<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
        <i>作者：
            <b>
            <a href="mailto:yangsw@dp.tech">杨舒文 📨 </a>
            </b>
        </i>
        <br/>
        <i>日期：2023-06-29</i><br/>
        <i>共享协议：</a>本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i><br/>
        <i>快速开始：点击上方的</i> <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;">开始连接</span> <i>按钮，选择 <b><u>bohrium-notebook:2023-05-31</u>镜像</b> 和任意配置机型即可开始。
    </p>
</div>

运行本notebook前，请确保已安装rdkit：
```shell
pip install rdkit
```

# PhysChem：融合物理/化学信息的分子性质预测
<a href="https://bohrium.dp.tech/notebook/879b6a3a9d394448913ca1ce7de257ec" target="_blank"><img src="https://cdn.dp.tech/bohrium/web/static/images/open-in-bohrium.svg" alt="Open In Bohrium"/></a>

## 1. 简介

近来，深度学习和分子科学领域的研究人员对二者的交叉点（AI4Science）产生了浓厚的兴趣，并在分子性质预测、分子图生成和药物发现的虚拟筛选等应用方面取得了显著进展。**分子表示学习**（molecular representation learning）旨在将分子的输入符号编码为数值向量（fingerprint），后者可作为下游任务的特征。早期的深度分子表示方法通常使用现成的网络架构，包括MPNN、GAT和Transformer。这些方法将分子的**线性符号**（例如SMILES）或**图符号**（即结构式）作为输入，而忽略了分子的**物理**和**化学**性质。

为了推动分子表示学习的边界，本工作从**物理**和**化学**两个角度重新审视了分子：
- **物理学家**一般将分子视为遵循（量子）力学规律的粒子系统。分子的主导构象反映了这些微观力学系统的平衡状态，因此受到广泛关注。
- **化学家**更关注分子的化学键和功能基团，这些键和基团表示电子的相互作用并决定化学/生物性质，如溶解度和毒性等。

然而，分子的物理和化学信息并非正交的。例如，键长和角度的扭转会极大影响粒子系统的动力学。因此，理想的分子表示不仅能够捕获物理和化学信息，还能适当地融合这两种信息。基于这种观念，作者提出了PhysChem，一种能够捕获和融合分子的物理和化学信息的新颖神经架构。PhysChem由两个专业网络组成：
- **物理学家网络**（PhysNet）通过在广义空间中模拟分子动力学来学习分子的主导构象。在PhysNet中，通过编码输入特征来初始化原子的隐含位置和动量。通过神经网络学习原子对之间的力，根据这些力，系统按照经典力学的规律运动。通过空间不变损失将最终的原子位置进行监督。
- **化学家网络**（ChemNet）利用消息传递框架来捕获原子和化学键的化学特征。ChemNet从原子状态和局部几何中生成消息，然后更新原子和化学键的状态。输出分子表示是从原子状态合并并通过标记的化学/生物医学性质进行监督。

除了专注于各自的专业领域外，这两个网络还通过共享专业知识来合作：PhysNet向ChemNet咨询化学键的隐藏表示以生成扭转力，而ChemNet利用PhysNet中间构象的局部几何。与现有方法相比，PhysChem具有两个优势：
1. _针对分子的物理/化学进行深度理解并融会贯通_，并设计对应的可解释性架构，以实现对分子物理/化学信息的处理。
2. _可以从头学习分子构象_，因此不需要给定初始构象，这扩展了PhysChem在无法获得这些标签的情况下的适用性，例如与药物筛选。

基于MoleculeNet以及SARS-CoV-2数据集的实验表明，PhysChem在**物理构型生成**和**化学性质预测**任务上均具有很好性能，取得了“1+1>2”的效果。

论文链接：[OpenReview-PhysChem](https://openreview.net/forum?id=Uxi7X1EqywV)

GitHub链接：[PKUterran/PhysChem](https://github.com/PKUterran/PhysChem)

## 2. 方法

![](image/arch.png)

如上图所示，PhysChem的结构分为四个部分：
1. **Initializer**将输入的分子图（支持PDB/SMILES格式）编码为初始atom/bond states $v^{(0)},e^{(0)}$（原子和化学键的表示向量，for ChemNet）以及初始的atomic positions/momenta $q^{(0)},p^{(0)}$（原子的三维坐标和动量，for PhysNet）。
2. **PhysNet Block**在高维隐空间中模拟分子运动。
3. **ChemNet Block**在原子和化学键之间执行几何感知的消息传递（geometry-aware message-passing）。

> - PhysNet Block和ChemNet Block会同步执行$L$次
> - 为确保物理引擎的功能一致性，每个PhysNet Block之间共享参数（ChemNet Block同理）
> - 为强化信息收集和表达能力，ChemNet Block内部不同层之间不共享参数

4. **Readout**模块根据下游任务的需求构造模型输出，如：
    - 物理构型：从原子隐式坐标$q^{(L)}$中读出三位欧氏空间坐标
    - 化学性质：从原子深层表示$v^{(L)}$中读出预测目标（分类/回归）

## 3. 功能

这里，我们提供了一个在QM9数据集上训练了两天的模型。你可以先<font color=red>配置一些参数</font>，然后加载它：

In [1]:
## 可以配置的参数有这些：
seed = 0
use_cuda = False


import torch
from functools import reduce
from data.encode import num_atom_features, num_bond_features
from train.config import QM9_CONFIG
from net.models import GeomNN as PhysChem
from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*')

tag = "QM9"
config = QM9_CONFIG.copy()
print('##### CONFIG #####')
for k, v in config.items():
    print(f'\t{k}: {v}')

model = PhysChem(
    atom_dim=num_atom_features(),
    bond_dim=num_bond_features(),
    config=config,
    use_cuda=use_cuda
)
model_dicts = torch.load(f'train/models/{tag}-model.pkl', map_location=torch.device('cpu'))
model.load_state_dict(model_dicts)
model.eval()

print('##### PARAMETERS #####')
param_size = 0
for name, param in model.named_parameters():
    print(f'\t{name}: {param.shape}')
    param_size += reduce(lambda x, y: x * y, param.shape)
print(f'Number of parameters: {param_size}')

##### CONFIG #####
	CLASSIFIER_HIDDENS: []
	INIT_GCN_H_DIMS: [128]
	INIT_GCN_O_DIM: 128
	INIT_LSTM_LAYERS: 2
	INIT_LSTM_O_DIM: 128
	HV_DIM: 128
	HE_DIM: 64
	HM_DIM: 256
	MV_DIM: 128
	ME_DIM: 64
	MM_DIM: 256
	PQ_DIM: 3
	N_LAYER: 2
	N_HOP: 1
	N_ITERATION: 4
	N_GLOBAL: 2
	MESSAGE_TYPE: triplet
	UNION_TYPE: gru
	GLOBAL_TYPE: inductive
	DERIVATION_TYPE: newton
	TAU: 0.25
	DISSA: 1.0
	DROPOUT: 0.5
	EPOCH: 300
	BATCH: 20
	PACK: 1
	CONF_LOSS: H_ADJ3
	LAMBDA: 100
	LR: 2e-06
	GAMMA: 0.995
	DECAY: 1e-05
	CONF_TYPE: ConfType.NEWTON
##### PARAMETERS #####
	initializer.v_linear.weight: torch.Size([128, 34])
	initializer.v_linear.bias: torch.Size([128])
	initializer.e_linear.weight: torch.Size([64, 10])
	initializer.e_linear.bias: torch.Size([64])
	initializer.a_linear.weight: torch.Size([1, 64])
	initializer.a_linear.bias: torch.Size([1])
	initializer.gcn.linears.0.weight: torch.Size([128, 128])
	initializer.gcn.linears.0.bias: torch.Size([128])
	initializer.gcn.linears.1.weight: torch.Size([128, 12

如果顺利的话，你已经成功加载一个PhysChem模型了，现在我们来看看它的输入和输出吧：

In [2]:
import rdkit.Chem as Chem
from train.utils.cache_batch import produce_batches_from_mols

mol = Chem.MolFromSmiles("CC")
batches = produce_batches_from_mols([mol])
batch = batches[0]

fingerprint, conformations, *_ = model.forward(
    atom_ftr=batch.atom_ftr,
    bond_ftr=batch.bond_ftr,
    massive=batch.massive,
    mask_matrices=batch.mask_matrices,
    return_derive=True,
)

	Start encoding...
	Encoded: 1


这个是分子指纹（molecular fingerprint），也就是表示一个分子的特征向量：

In [3]:
print(fingerprint.shape)

torch.Size([1, 256])


这个是分子在构型演化过程中，每个时间步的原子三维坐标：

In [4]:
print(len(conformations))
print(conformations[-1].detach().numpy())

10
[[-1.2077603 -1.8745091  1.7269976]
 [-0.6036631 -2.7950263  2.8480675]]


### 3.1. 物理构型生成

在这部分中，我们将展示如何使用PhysChem为分子生成物理构型：

In [14]:
list_smiles = [
    'O=Cc1cc(C#N)ccc1',
    'C(C(C(=O)O)N)C(=O)O',
    'CC(=O)OC1=CC=CC=C1C(=O)O',
]

mols = [Chem.MolFromSmiles(list_smiles[i]) for i in range(len(list_smiles))]
batches = produce_batches_from_mols(mols)
list_physchem_conformations = []
for batch in batches:
    _, conformations, *_ = model.forward(
        atom_ftr=batch.atom_ftr,
        bond_ftr=batch.bond_ftr,
        massive=batch.massive,
        mask_matrices=batch.mask_matrices,
        return_derive=True,
    )
    list_physchem_conformations.append([conf.detach().numpy() for conf in conformations])

	Start encoding...
	Encoded: 3


顺便用RDKit和[HamEng](https://openreview.net/forum?id=q-cnWaaoUTH)生成构型：

In [15]:
from train.utils.rdkit import rdkit_mol_positions

list_rdkit_conformation = [rdkit_mol_positions(mol) for mol in mols]

In [16]:
from net.models import MLP
from net.baseline.HamEng.models import HamiltonianPositionProducer
from train.HamEng.config import FITTER_CONFIG_QM9 as HAMENG_CONFIG
from visualize.vis_derive import generate_derive

config = HAMENG_CONFIG.copy()
hameng_model = HamiltonianPositionProducer(
    n_dim=num_atom_features(),
    e_dim=num_bond_features(),
    config=config,
    use_cuda=use_cuda
)
conf_gen = MLP(
    in_dim=config['PQ_DIM'],
    out_dim=3,
    use_cuda=use_cuda,
    bias=False
)

model_dicts = torch.load(f'train/models/HamNet/HamEng@16880611-model.pkl', map_location=torch.device('cpu'))
conf_dicts = torch.load(f'train/models/HamNet/HamEng@16880611-conf_gen.pkl', map_location=torch.device('cpu'))
hameng_model.load_state_dict(model_dicts)
conf_gen.load_state_dict(conf_dicts)
model.eval()
conf_gen.eval()

list_hameng_conformations = []
for batch in batches:
    _, list_q_ftr, *_ = hameng_model.forward(
        v_features=batch.atom_ftr,
        e_features=batch.bond_ftr,
        massive=batch.massive,
        mask_matrices=batch.mask_matrices,
        return_multi=True
    )
    list_hameng_conformations.append([conf_gen.forward(q).detach().numpy() for q in list_q_ftr])

然后对比这些模型生成的物理构型：

In [ ]:
import numpy as np
import py3Dmol
import rdkit.Chem as Chem
from rdkit.Chem.rdchem import Mol as Molecule
from rdkit.Chem.AllChem import EmbedMolecule
from train.utils.kabsch import kabsch_np

def show_pdb(pdb: str):
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb)
    view.setStyle({'stick': {}})
    view.setStyle({'model': 0}, {'stick': {'colorscheme': 'cyanCarbon'}})
    view.zoomTo()
    view.show()

def show_mol_with_pos(mol: Molecule, pos: np.ndarray):
    EmbedMolecule(mol)
    for i in range(len(mol.GetAtoms())):
        mol.GetConformer().SetAtomPosition(i, [float(p) for p in pos[i]])
    # mol = AddHs(mol)
    Chem.MolToPDBFile(mol, filename=f'visualize/temp.pdb')
    with open(f'visualize/temp.pdb') as fp:
        pdb = fp.read()
    show_pdb(pdb)

for i, smiles in enumerate(list_smiles):
    # alignment
    rdkit_conf = list_rdkit_conformation[i]
    hameng_confs = list_hameng_conformations[i]
    physchem_confs = list_physchem_conformations[i]
    hameng_confs = [kabsch_np(conf, rdkit_conf)[0] for conf in hameng_confs]
    physchem_confs = [kabsch_np(conf, rdkit_conf)[0] for conf in physchem_confs]

    print(f'##### Generating SMILES {smiles} #####')
    print(f"\tRDKit:")
    show_mol_with_pos(mols[i], rdkit_conf)
    print(f"\tHamEng:")
    show_mol_with_pos(mols[i], hameng_confs[-1])
    print(f"\tPhysChem:")
    show_mol_with_pos(mols[i], physchem_confs[-1])

### 3.2 化学性质预测

在这部分中，我们将展示如何使用PhysChem为分子生成fingerprint，并预测化学性质：（以`FreeSolv`为例）

In [38]:
import numpy as np
from typing import List, Tuple
from tqdm import tqdm
from data.FreeSolv.load_freesolv import load_freesolv
from train.utils.cache_batch import Batch
from train.utils.cache_batch import load_encode_mols, load_batch_cache, batch_cuda_copy

data_name = "FreeSolv"
mols, mol_properties = load_freesolv()
mols_info = load_encode_mols(mols, name=data_name)
mean_p = np.mean(mol_properties, axis=0)
stddev_p = np.std((mol_properties - mean_p).tolist(), axis=0, ddof=1)
print(f'\tmean: {mean_p[0]}')
print(f'\tstd: {stddev_p[0]}')
norm_p = (mol_properties - mean_p) / stddev_p
print('Caching Batches...')
batch_cache = load_batch_cache(
    data_name, mols, mols_info, norm_p, batch_size=config['BATCH'],
    contains_ground_truth_conf=False, use_cuda=use_cuda, use_tqdm=True
)

def proc_batch(batches: List[Batch]) -> Tuple[np.ndarray, np.ndarray]:
    list_fp = []
    list_prop = []
    batches = tqdm(batches, total=len(batches))
    for batch in batches:
        if use_cuda:
            batch = batch_cuda_copy(batch)
        fp, *_ = model.forward(batch.atom_ftr, batch.bond_ftr, batch.massive, batch.mask_matrices)
        list_fp.append(fp.cpu().detach().numpy())
        list_prop.append(batch.properties.cpu().detach().numpy())
    return np.vstack(list_fp), np.vstack(list_prop)

train_fp, train_prop = proc_batch(batch_cache.train_batches)
valid_fp, valid_prop = proc_batch(batch_cache.validate_batches)
test_fp, test_prop = proc_batch(batch_cache.test_batches)

	Use Cached Mols
	mean: -3.803006172180176
	std: 3.8478201122287583
Caching Batches...
	Use Cached Batches


100%|██████████| 4/4 [00:00<00:00,  4.81it/s]


In [43]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

reg = SGDRegressor()
reg.fit(train_fp, train_prop[:, 0])
train_pred = reg.predict(train_fp)
valid_pred = reg.predict(valid_fp)
test_pred = reg.predict(test_fp)
print(f"TRAIN RMSE: {mean_squared_error(train_prop, train_pred) ** 0.5 * stddev_p[0]}")
print(f"VALIDATE RMSE: {mean_squared_error(valid_prop, valid_pred) ** 0.5 * stddev_p[0]}")
print(f"TEST RMSE: {mean_squared_error(test_prop, test_pred) ** 0.5 * stddev_p[0]}")

TRAIN RMSE: 2.0468615127731553
VALIDATE RMSE: 2.763132534140123
TEST RMSE: 1.794972227138338


## 4. 训练自己的模型

我们提供了针对多种输出类型的训练函数：
- `train/single_regression.py`：单回归问题，如Lipop，ESOL，FreeSolv
- `train/train_multi_classification.py`：多分类问题，如Sars-Cov-2
- `train/train_qm9.py`：多回归+构型预测问题，如QM7/8/9

例如，在QM7上训练模型的方法如下：

In [ ]:
from net.config import ConfType
from train.train_qm9 import train_qm9, QMDataset

conf_type = ConfType.NEWTON
name = 'QM7'
seed = 16880611

train_qm9(
    special_config={
        'CLASSIFIER_HIDDENS': [],

        'INIT_GCN_H_DIMS': [128],
        'INIT_GCN_O_DIM': 128,
        'INIT_LSTM_LAYERS': 2,
        'INIT_LSTM_O_DIM': 128,

        'HV_DIM': 200,
        'HE_DIM': 100,
        'HM_DIM': 300,
        'MV_DIM': 200,
        'ME_DIM': 100,
        'MM_DIM': 300,
        'PQ_DIM': 3,
        'N_LAYER': 2,
        'N_HOP': 1,
        'N_ITERATION': 4,
        'N_GLOBAL': 2,
        'MESSAGE_TYPE': 'triplet',
        'UNION_TYPE': 'gru',
        'GLOBAL_TYPE': 'inductive',
        'DERIVATION_TYPE': 'newton',
        'TAU': 0.25,
        'DISSA': 1.0,
        'DROPOUT': 0.5,

        'EPOCH': 100,
        'BATCH': 10,
        'PACK': 10,
        'CONF_LOSS': 'H_ADJ3',
        'LAMBDA': 0.1,
        'LR': 1e-4,
        'GAMMA': 0.95,
        'DECAY': 1e-3,

        'CONF_TYPE': conf_type,
    },
    dataset=QMDataset.QM7,
    use_cuda=True,
    max_num=-1,
    data_name=f'{name}@{seed}',
    seed=seed,
    force_save=True,
    tag=f'{name}@{seed}',
    use_tqdm=False,
)